In [ ]:
#@title pip install
!pip install ipywidgets

In [ ]:
#@title Import
import random
import time
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, VBox, HBox, HTML, Layout, Dropdown

In [ ]:
#@title Mindmap Simulator
class StakeholderMindMapSimulator:
    def __init__(self):
        self.scenarios = [
            {
                "title": "Hiring Algorithm Accuracy Gap",
                "description": "Your hiring algorithm shows 85% accuracy for men and 78% accuracy for women. The 7% gap has created internal debate about next steps.",
                "stakeholders": {
                    "Legal": {
                        "position": "Immediate demographic parity required",
                        "concern": "Legal liability",
                        "priority": "Risk avoidance",
                        "stance": "Fix-focused"
                    },
                    "Engineering": {
                        "position": "7% gap is acceptable variation",
                        "concern": "Technical integrity",
                        "priority": "Accuracy maintenance",
                        "stance": "Status-quo"
                    },
                    "HR": {
                        "position": "Gradual change with training",
                        "concern": "Implementation feasibility",
                        "priority": "Change management",
                        "stance": "Balanced"
                    },
                    "Diversity": {
                        "position": "Prioritize equal outcomes",
                        "concern": "Historical inequity",
                        "priority": "Social justice",
                        "stance": "Fix-focused"
                    },
                    "Candidates": {
                        "position": "Transparent fair process",
                        "concern": "Process clarity",
                        "priority": "Understanding decisions",
                        "stance": "Transparency-focused"
                    },
                    "Leadership": {
                        "position": "Balance talent quality and legal risk",
                        "concern": "Business outcomes",
                        "priority": "Competitive advantage",
                        "stance": "Business-focused"
                    }
                },
                "connections": {
                    "Legal-Diversity": {"type": "Strong alignment", "reason": "Both want immediate action"},
                    "Engineering-Leadership": {"type": "Moderate alignment", "reason": "Both prioritize performance"},
                    "HR-Leadership": {"type": "Moderate alignment", "reason": "Both want sustainable solutions"},
                    "Legal-Engineering": {"type": "Strong tension", "reason": "Immediate fix vs. technical concerns"},
                    "Diversity-Engineering": {"type": "Strong tension", "reason": "Social justice vs. technical merit"},
                    "Candidates-HR": {"type": "Moderate alignment", "reason": "Both want clear communication"}
                }
            },
            {
                "title": "Medical AI Racial Accuracy Disparity",
                "description": "A diagnostic AI shows 92% accuracy for white patients and 78% accuracy for Black patients.",
                "stakeholders": {
                    "Ethics Board": {
                        "position": "Suspend until equal accuracy",
                        "concern": "Patient safety",
                        "priority": "Do no harm",
                        "stance": "Safety-first"
                    },
                    "Administration": {
                        "position": "Continue with improvements",
                        "concern": "Overall patient care",
                        "priority": "Operational continuity",
                        "stance": "Pragmatic"
                    },
                    "AI Team": {
                        "position": "Systematic data collection needed",
                        "concern": "Technical root cause",
                        "priority": "Sustainable solution",
                        "stance": "Technical"
                    },
                    "Patient Advocates": {
                        "position": "Immediate action and transparency",
                        "concern": "Community trust",
                        "priority": "Health equity",
                        "stance": "Advocacy"
                    },
                    "Physicians": {
                        "position": "Clear accuracy reporting by demographics",
                        "concern": "Clinical decisions",
                        "priority": "Informed practice",
                        "stance": "Information-focused"
                    }
                },
                "connections": {
                    "Ethics Board-Patient Advocates": {"type": "Strong alignment", "reason": "Both demand immediate action"},
                    "Administration-AI Team": {"type": "Moderate alignment", "reason": "Both want sustainable solutions"},
                    "Physicians-Patient Advocates": {"type": "Moderate alignment", "reason": "Both want transparency"},
                    "Ethics Board-Administration": {"type": "Strong tension", "reason": "Suspend vs. continue debate"},
                    "AI Team-Patient Advocates": {"type": "Moderate tension", "reason": "Long-term fix vs. immediate action"},
                    "Physicians-Administration": {"type": "Moderate alignment", "reason": "Both want practical solutions"}
                }
            }
        ]

        self.current_scenario = 0
        self.revealed_connections = set()

        # Create widgets using ipywidgets.HTML
        self.scenario_display = HTML(value="<h3>Stakeholder Mind Map Simulator</h3>")

        self.mindmap_display = HTML(value="<div>Mind map will appear here</div>")

        self.connection_button = Button(
            description='Reveal Connections',
            button_style='info',
            layout=Layout(width='200px', height='40px', margin='5px')
        )

        self.analyze_button = Button(
            description='Analyze Patterns',
            button_style='warning',
            layout=Layout(width='200px', height='40px', margin='5px')
        )

        self.next_button = Button(
            description='Next Scenario →',
            button_style='',
            layout=Layout(width='150px', height='30px', margin='10px')
        )

        self.insights_display = HTML(value="")

        # Button clicks
        self.connection_button.on_click(lambda b: self.reveal_connections())
        self.analyze_button.on_click(lambda b: self.analyze_patterns())
        self.next_button.on_click(lambda b: self.next_scenario())

        # Layout
        button_box = HBox(
            [self.connection_button, self.analyze_button, self.next_button],
            layout=Layout(justify_content='center')
        )

        self.widget = VBox([
            self.scenario_display,
            self.mindmap_display,
            button_box,
            self.insights_display
        ])

    def start_mindmap_simulation(self):
        self.current_scenario = 0
        self.revealed_connections = set()
        self.show_scenario()
        return self.widget

    def show_scenario(self):
        if self.current_scenario >= len(self.scenarios):
            self.show_final_insights()
            return

        scenario = self.scenarios[self.current_scenario]
        self.revealed_connections = set()

        self.scenario_display.value = f"""
        <div style='padding: 20px; background: #f9f9f9; border-radius: 10px; margin: 10px 0; text-align: center;'>
            <h3>Scenario {self.current_scenario + 1}: {scenario['title']}</h3>
            <p style='font-size: 16px;'><strong>{scenario['description']}</strong></p>
            <p style='color: #666; margin-top: 15px;'><em><strong>💡 Challenge:</strong> Study each stakeholder's position below. Try to predict which stakeholders might align and which might be in tension with each other. Think about their priorities, concerns, and timelines before revealing any connections!</em></p>
        </div>
        """

        self.show_mindmap()
        self.insights_display.value = ""

    def show_mindmap(self):
        scenario = self.scenarios[self.current_scenario]
        stakeholders = scenario['stakeholders']

        # Create SVG mind map
        svg_width = 800
        svg_height = 500
        center_x = svg_width // 2
        center_y = svg_height // 2

        # Position stakeholders in a circle around the center
        import math
        num_stakeholders = len(stakeholders)
        radius = 180

        svg_content = f'''
        <svg width="{svg_width}" height="{svg_height}" viewBox="0 0 {svg_width} {svg_height}">
            <!-- Central issue -->
            <circle cx="{center_x}" cy="{center_y}" r="60" fill="white" stroke="#333" stroke-width="2"/>
            <text x="{center_x}" y="{center_y-5}" text-anchor="middle" font-size="12" font-weight="bold" fill="black">FAIRNESS</text>
            <text x="{center_x}" y="{center_y+10}" text-anchor="middle" font-size="12" font-weight="bold" fill="black">ISSUE</text>
        '''

        stakeholder_positions = {}
        for i, (name, data) in enumerate(stakeholders.items()):
            angle = (2 * math.pi * i) / num_stakeholders
            x = center_x + radius * math.cos(angle)
            y = center_y + radius * math.sin(angle)
            stakeholder_positions[name] = (x, y)

            # All stakeholders dark grey with white text
            color = "#555555"

            # Draw larger stakeholder node with center-aligned text
            svg_content += f'''
            <circle cx="{x}" cy="{y}" r="55" fill="{color}" stroke="#333" stroke-width="2" opacity="0.9"/>
            <text x="{x}" y="{y-18}" text-anchor="middle" font-size="11" font-weight="bold" fill="white">{name}</text>
            <text x="{x}" y="{y-5}" text-anchor="middle" font-size="9" fill="white">{data['concern']}</text>
            <text x="{x}" y="{y+8}" text-anchor="middle" font-size="9" font-style="italic" fill="white">{data['priority']}</text>
            '''

        # Add connection lines if revealed
        if self.revealed_connections:
            connections = scenario['connections']
            for connection_key in self.revealed_connections:
                if connection_key in connections:
                    connection = connections[connection_key]
                    stakeholder1, stakeholder2 = connection_key.split('-')

                    if stakeholder1 in stakeholder_positions and stakeholder2 in stakeholder_positions:
                        x1, y1 = stakeholder_positions[stakeholder1]
                        x2, y2 = stakeholder_positions[stakeholder2]

                        # Line style based on connection type
                        if "alignment" in connection['type']:
                            stroke_color = "#2ECC71"
                        else:  # tension
                            stroke_color = "#E74C3C"

                        svg_content += f'''
                        <line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}"
                              stroke="{stroke_color}" stroke-width="3"
                              opacity="0.8" stroke-dasharray="5,5"/>
                        '''

        svg_content += '</svg>'

        # Simplified legend
        legend = '''
        <div style="margin-top: 10px; text-align: center;">
            <p style="color: #666;"><strong>Study each stakeholder's concerns and priorities. Can you predict their relationships?</strong></p>
        '''

        if self.revealed_connections:
            legend += '''
            <div style="margin-top: 10px; text-align: center;">
                <p><strong>Connections:</strong></p>
                <span style="color: #2ECC71;">━━━</span> Alignment &nbsp;&nbsp;
                <span style="color: #E74C3C;">━━━</span> Tension
            </div>
            '''

        legend += '</div>'

        self.mindmap_display.value = f'<div style="text-align: center;">{svg_content}{legend}</div>'

    def reveal_connections(self):
        scenario = self.scenarios[self.current_scenario]
        connections = scenario['connections']

        # Reveal one connection at a time
        unrevealed = set(connections.keys()) - self.revealed_connections
        if unrevealed:
            next_connection = list(unrevealed)[0]
            self.revealed_connections.add(next_connection)
            self.show_mindmap()

            # Show insight about this connection
            connection_data = connections[next_connection]
            stakeholder1, stakeholder2 = next_connection.split('-')

            self.insights_display.value = f'''
            <div style="border: 2px solid #3498DB; padding: 15px; border-radius: 8px; margin: 10px 0; background: #EBF3FD; text-align: center;">
                <h4 style="color: #2980B9; margin: 0;">🔗 Connection Revealed: {stakeholder1} ↔ {stakeholder2}</h4>
                <p><strong>Relationship:</strong> {connection_data['type']}</p>
                <p><strong>Reason:</strong> {connection_data['reason']}</p>
                <p style="color: #666; font-style: italic;">Did you predict this relationship correctly?</p>
            </div>
            '''
        else:
            self.insights_display.value = '''
            <div style="border: 2px solid #2ECC71; padding: 15px; border-radius: 8px; margin: 10px 0; background: #E8F8F5; text-align: center;">
                <p><strong>All connections revealed!</strong> Click "Analyze Patterns" to see the bigger picture.</p>
            </div>
            '''

    def analyze_patterns(self):
        scenario = self.scenarios[self.current_scenario]
        connections = scenario['connections']

        # Analyze alignment vs tension patterns
        alignments = [c for c in connections.values() if "alignment" in c['type']]
        tensions = [c for c in connections.values() if "tension" in c['type']]

        analysis = f'''
        <div style="border: 2px solid #9B59B6; padding: 20px; border-radius: 10px; margin: 10px 0; background: #F4F1F8; text-align: center;">
            <h4 style="color: #8E44AD;">🧠 Pattern Analysis</h4>

            <h5>Connection Summary:</h5>
            <ul style="text-align: left; display: inline-block;">
                <li><strong>Alignments:</strong> {len(alignments)} connections</li>
                <li><strong>Tensions:</strong> {len(tensions)} connections</li>
            </ul>

            <h5>Key Insight:</h5>
            <p style="background: #FFFFCC; padding: 10px; border-radius: 5px; font-style: italic;">
            <strong>💡 Skill Building:</strong> Learning to predict stakeholder relationships helps you navigate complex ethical decisions.
            Notice how stakeholders with similar timelines and concerns tend to align, while those with different priorities create tension.
            This pattern recognition is crucial for building consensus and managing conflicts in real situations.
            </p>
        </div>
        '''

        self.insights_display.value = analysis

    def next_scenario(self):
        self.current_scenario += 1
        self.show_scenario()

    def show_final_insights(self):
        self.scenario_display.value = '''
        <div style="border: 3px solid #4CAF50; padding: 25px; border-radius: 15px; background: #f8fff8; text-align: center;">
            <h2 style="color: #4CAF50;">Mind Map Analysis Complete! 🧠</h2>

            <h3>Developed Skill: Stakeholder Pattern Recognition</h3>
            <p>You've practiced predicting and analyzing how different stakeholders relate to each other around fairness issues.
            This prediction skill helps you identify natural coalitions, anticipate conflicts, and find common ground before entering difficult conversations.</p>

            <h4>Key Insights:</h4>
            <ul style="text-align: left; max-width: 600px; margin: 0 auto;">
                <li><strong>Prediction Practice:</strong> Anticipating relationships helps you prepare for stakeholder dynamics</li>
                <li><strong>Pattern Recognition:</strong> Similar concerns and timelines often predict alignment</li>
                <li><strong>Tension Sources:</strong> Conflicts arise from competing values, not just different opinions</li>
                <li><strong>Strategic Thinking:</strong> Understanding relationships helps you build coalitions and manage conflicts</li>
            </ul>

            <p style="margin-top: 20px;"><strong>This predictive approach builds the meta-skill of anticipating stakeholder dynamics
            before they unfold in real situations.</strong></p>
        </div>
        '''

        self.mindmap_display.value = ""
        self.insights_display.value = ""

In [ ]:
#@title Start Simulator
# Start the Stakeholder Mind Map Simulator
simulator = StakeholderMindMapSimulator()
widget = simulator.start_mindmap_simulation()
display(widget)